In [12]:
import os
from datetime import timedelta, datetime
import talib
import pandas as pd
from sqlalchemy import create_engine
import dotenv

In [13]:
dotenv.load_dotenv()
SQLALCHEMY_DATABASE_URI = os.getenv('SQLALCHEMY_DATABASE_URI')

In [14]:
db_engine = create_engine(
    SQLALCHEMY_DATABASE_URI, 
    pool_size=10, 
    max_overflow=20,
    )

In [15]:
request_dict = {
    'symbol': 'ICICIBANK',
    'dates': {
        'start': '2018-01-01',
        'end': '2019-01-01'
    }
}

In [16]:
df = pd.read_sql('''
    SELECT *
    FROM stock_daily_data
    WHERE symbol = '{}'
    AND   TIMESTAMP BETWEEN '{}' AND '{}'
    ORDER BY TIMESTAMP
    LIMIT 1000;
    '''.format(
            request_dict['symbol'],
            datetime.strptime(request_dict['dates']['start'], '%Y-%m-%d') - timedelta(days=500),
            request_dict['dates']['end'],
        ), db_engine, index_col=['timestamp']);

In [17]:
'''
# Generate CSV
df.to_csv('CSV/{}_{}_{}.csv'.format(
    request_dict['symbol'],
    request_dict['dates']['start'],
    request_dict['dates']['end'],
))
'''
# Preview
df

,Open,High,Low,Close,Volume,symbol,updated_at
timestamp,,,,,,,
2016-08-19,230.000,232.455,228.682,230.818,14519513,ICICIBANK,2020-04-17 03:48:47
2016-08-22,231.136,231.773,226.091,229.318,9076842,ICICIBANK,2020-04-17 03:48:47
2016-08-23,227.682,229.091,226.136,227.455,10995821,ICICIBANK,2020-04-17 03:48:47
2016-08-24,227.455,228.818,225.318,225.864,9228359,ICICIBANK,2020-04-17 03:48:47
2016-08-25,225.409,227.455,223.727,224.500,14901359,ICICIBANK,2020-04-17 03:48:47
...,...,...,...,...,...,...,...
2018-12-26,350.650,356.900,348.150,355.850,8695108,ICICIBANK,2020-04-16 23:54:18
2018-12-27,359.500,360.750,354.900,355.650,10122458,ICICIBANK,2020-04-16 23:54:18
2018-12-28,358.300,362.800,358.300,360.750,13004606,ICICIBANK,2020-04-16 23:54:18


In [18]:
# calculating indicators now
df['EMA50'] = talib.EMA(df.Close, timeperiod=50)
df['EMA200'] = talib.EMA(df.Close, timeperiod=200)
df['rsa14'] = talib.RSI(df.Close, timeperiod=14)
df['sar'] = talib.SAR(
    df.High,
    df.Low,
    acceleration=0.02, 
    maximum=0.2,
    )
df['macd'], df['macdsignal'], df['macdhist'] = talib.MACDEXT(
        df.Close, 
        fastperiod=12, 
        fastmatype=0, 
        slowperiod=26, 
        slowmatype=0, 
        signalperiod=9, 
        signalmatype=0,
        )
df['slowk'], df['slowkd'] = talib.STOCH(
        df.High, df.Low, df.Close,
        fastk_period=14, 
        slowk_period=3, 
        slowk_matype=0, 
        slowd_period=3, 
        slowd_matype=0,
        )
df['upperband'], df['middleband'], df['lowerband'] = talib.BBANDS(
        df.Close, 
        timeperiod=5, 
        nbdevup=2, 
        nbdevdn=2, 
        matype=0,
        )

In [19]:
# Preview
df                          

,Open,High,Low,Close,Volume,symbol,updated_at,EMA50,EMA200,rsa14,sar,macd,macdsignal,macdhist,slowk,slowkd,upperband,middleband,lowerband
timestamp,,,,,,,,,,,,,,,,,,,
2016-08-19,230.000,232.455,228.682,230.818,14519513,ICICIBANK,2020-04-17 03:48:47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-08-22,231.136,231.773,226.091,229.318,9076842,ICICIBANK,2020-04-17 03:48:47,NaN,NaN,NaN,232.455000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-08-23,227.682,229.091,226.136,227.455,10995821,ICICIBANK,2020-04-17 03:48:47,NaN,NaN,NaN,232.327720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-08-24,227.455,228.818,225.318,225.864,9228359,ICICIBANK,2020-04-17 03:48:47,NaN,NaN,NaN,232.202986,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-08-25,225.409,227.455,223.727,224.500,14901359,ICICIBANK,2020-04-17 03:48:47,NaN,NaN,NaN,231.927586,NaN,NaN,NaN,NaN,NaN,232.147434,227.591,223.034566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-26,350.650,356.900,348.150,355.850,8695108,ICICIBANK,2020-04-16 23:54:18,347.552914,321.142767,52.038475,347.712060,-0.756410,-3.056268,2.299858,54.937343,63.742690,368.708330,358.290,347.871670
2018-12-27,359.500,360.750,354.900,355.650,10122458,ICICIBANK,2020-04-16 23:54:18,347.870447,321.486122,51.851112,348.150000,0.278205,-2.646296,2.924501,56.390977,57.527151,362.551050,356.120,349.688950
2018-12-28,358.300,362.800,358.300,360.750,13004606,ICICIBANK,2020-04-16 23:54:18,348.375527,321.876808,56.183461,348.150000,1.664423,-2.021688,3.686111,64.411028,58.579783,361.234961,355.840,350.445039


In [ ]:
# Generate CSV
df.to_csv('CSV/{}_{}_{}_indicators.csv'.format(
    request_dict['symbol'],
    request_dict['dates']['start'],
    request_dict['dates']['end'],
))